# Midterm 2

## FINM 36700 - 2024

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

* Name: Jose Luna 
* Disclaimer: I used my own code and also utilized GPT and Copilot for syntax and editing corrections.

# Instructions

## Please note the following:

Points
* The exam is 100 points.
* You have 120 minutes to complete the exam.
* For every minute late you submit the exam, you will lose one point.


Submission
* You will upload your solution to the `Midterm 2` assignment on Canvas, where you downloaded this. 
* Be sure to **submit** on Canvas, not just **save** on Canvas.
* Your submission should be readable, (the graders can understand your answers.)
* Your submission should **include all code used in your analysis in a file format that the code can be executed.** 

Rules
* The exam is open-material, closed-communication.
* You do not need to cite material from the course github repo - you are welcome to use the code posted there without citation.

Advice
* If you find any question to be unclear, state your interpretation and proceed. We will only answer questions of interpretation if there is a typo, error, etc.
* The exam will be graded for partial credit.

## Data

**All data files are found in the class github repo, in the `data` folder.**

This exam makes use of the following data files:
* `midterm_2_data.xlsx`

This file contains the following sheets:
- for Section 2:
    * `sector stocks excess returns` - MONTHLY excess returns for 49 sector stocks
    * `factors excess returns` - MONTHLY excess returns of AQR factor model from Homework 5
- for Section 3:
    * `factors excess returns` - MONTHLY excess returns of AQR factor model from Homework 5

## Scoring

| Problem | Points |
|---------|--------|
| 1       | 25     |
| 2       | 40     |
| 3       | 35     |

### Each numbered question is worth 5 points unless otherwise specified.

# 1. Short Answer

#### No Data Needed

These problems do not require any data file. Rather, analyze them conceptually. 

### 1.1.

Historically, which pricing factor among the ones we studied has shown a considerable decrease in importance?

*Answer* : 

The Size Factor (`SMB`) has shown a considerable decrease in importance historically. Initially highlighted in the Fama-French Three-Factor Model, the size factor represented the return difference between small-cap and large-cap stocks. However, empirical evidence and recent research indicate that the size factor's mean excess return has diminished over time, making it less significant in explaining asset returns. This is reflected in modern quantitative strategies (for instance, tangency portfolio in MV), where the size factor often receives minimal weight or is excluded altogether due to its low or uncertain excess returns.

### 1.2.

True or False: For a given factor model and a set of test assets, the addition of one more factor to that model will surely decrease the cross-sectional MAE. 

True or False: For a given factor model and a set of test assets, the addition of one more factor to that model will surely decrease the time-series MAE. 

Along with stating T/F, explain your reasoning for the two statements.

*Answer*: 

1. **True** : In cross-sectional tests, adding an additional factor introduces more parameters, which generally improves the model's fit to the data. The cross-sectional Mean Absolute Error (MAE) measures the average deviation of predicted asset returns from actual returns across different assets at a single point in time. By adding more factors, we will generally capture more of the variation in asset returns, thereby reducing the cross-sectional MAE. At least we impose contraints on the model.

2. **False**: In time-series tests, adding an additional factor does not guarantee a decrease in the MAE. In time-series regressions, the focus is on the alpha (intercept term) as a proxy of MAE. Adding more factors does not necessarily reduce the magnitude of the alpha, and in some cases, it might even increase the time-series MAE if the new factor does not align well with the asset's return dynamics. But it would improve the R-Squared of the model (which is not important in TS).

### 1.3.

Consider the scenario in which you are helping two people with investments.

* The young person has a 50 year investment horizon.
* The elderly person has a 10 year investment horizon.
* Both individuals have the same portfolio holdings.

State who has the more certain cumulative return and explain your reasoning.

*Answer* :

While the average (mean) return becomes more predictable over longer horizons due to the Law of Large Numbers, the variance of cumulative returns increases with time. This is because cumulative returns compound over time, and the uncertainty (volatility) associated with returns accumulates. For **log-normal returns**, the variance of cumulative returns scales linearly with the time horizon (h). Therefore, over a 50-year period, the young person faces greater uncertainty in cumulative returns compared to the elderly person over a 10-year period. The shorter time horizon results in less cumulative volatility, making the elderly person's cumulative return more certain.

### 1.4.

Suppose we find that the 10-year bond yield works well as a new pricing factor, along with `MKT`.

Consider two ways of building this new factor.
1. Directly use the index of 10-year yields, `YLD`
1. Construct a Fama-French style portfolio of equities, `FFYLD`. (Rank all the stocks by their correlation to bond yield changes, and go long the highest ranked and short the lowest ranked.)

Could you test the model with `YLD` and the model with `FFYLD` in the exact same ways? Explain.

*Answer* :

No, we cannot test the model with `YLD` and the model with `FFYLD` in the exact same ways. Since `YLD` is a economic factor (Non-return Factor), so we can not use Time Series, only a Cross Sectional model validation. On contrast, if we construct `FFYLD` following the FF style, it would be a return factor, and we can validate with either Time Series or Cross Sectional Validation.

### 1.5.

Suppose we implement a momentum strategy on cryptocurrencies rather than US stocks.

Conceptually speaking, but specific to the context of our course discussion, how would the risk profile differ from the momentum strategy of US equities?

*Answer* : 

The momentum strategy on cryptocurrencies would likely entail higher overall risk due to increased volatility and possible liquidity concerns. These factors contribute to a risk profile that is different and potentially more risk-taker than that of a momentum strategy on US equities.

***

# 2. Pricing and Tangency Portfolio

You work in a hedge fund that believes that the AQR 4-Factor Model (present in Homework 5) is the perfect pricing model for stocks.

$$
\mathbb{E} \left[ \tilde{r}^i \right] = \beta^{i,\text{MKT}} \mathbb{E} \left[ \tilde{f}_{\text{MKT}} \right] + \beta^{i,\text{HML}} \mathbb{E} \left[ \tilde{f}_{\text{HML}} \right] + \beta^{i,\text{RMW}} \mathbb{E} \left[ \tilde{f}_{\text{RMW}} \right] + \beta^{i,\text{UMD}} \mathbb{E} \left[ \tilde{f}_{\text{UMD}} \right]
$$

The factors are available in the sheet `factors excess returns`.

The hedge fund invests in sector-tracking ETFs available in the sheet `sectors excess returns`. You are to allocate into these sectors according to a mean-variance optimization with...

* regularization: elements outside the diagonal covariance matrix divided by 2.
* modeled risk premia: expected excess returns given by the factor model rather than just using the historic sample averages.

You are to train the portfolio and test out-of-sample. The timeframes should be:
* Training timeframe: Jan-2018 to Dec-2022.
* Testing timeframe: Jan-2023 to most recent observation.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew,kurtosis,norm
import statsmodels.api as sm

path = "../../data/midterm_2_data.xlsx"
data_sectors = pd.read_excel(path, sheet_name="sector excess returns", index_col=0)
data_factors = pd.read_excel(path, sheet_name="factors excess returns", index_col=0)

data_factors_training = data_factors[(data_factors.index >= "2018-01-01") & (data_factors.index < "2022-12-31")]
data_sectors_training = data_sectors[(data_sectors.index >= "2018-01-01") & (data_sectors.index < "2022-12-31")]
data_sectors_testing = data_sectors[(data_sectors.index >= "2023-01-01") ]
data_factors_testing = data_factors[(data_factors.index >= "2023-01-01") ]

In [3]:
def weights_tag_reg(return_db, adj_factor = 12, diagonal_factor = 1, denominator = 2):
    sigma = (return_db.cov()*adj_factor)
    sigma_reg = (sigma + diagonal_factor*np.diag(np.diag(sigma)))/denominator
    mu_excess = (return_db.mean()*adj_factor)
    vector = np.ones(len(mu_excess))
    w_tan = (np.linalg.inv(sigma_reg) @ mu_excess )/(np.transpose(vector) @ np.linalg.inv(sigma_reg) @ mu_excess)
    weights_db = pd.DataFrame({"w_tan_reg": w_tan})
    weights_db.index = return_db.columns
    return weights_db
def get_metric_returns(returns, weights=[], adj_factor=12, VaR_q=5):


    # If weights are provided, compute the weighted returns
    if len(weights) == 0:
        port_metrics = returns.copy()
    else:
        port_metrics = returns @ weights
        port_metrics = pd.DataFrame(port_metrics, columns=['Portfolio'])

    # Initialize the result DataFrame
    result = pd.DataFrame()

    # Compute Mean, Volatility, Sharpe Ratio, Skew, Excess Kurtosis
    if len(weights) == 0:
        port_metrics_r = pd.DataFrame({
            "Mean": port_metrics.mean() * adj_factor,
            "Volatility": port_metrics.std() * np.sqrt(adj_factor)
        })
        port_metrics_r["Sharpe_Ratio"] = (port_metrics.mean() / port_metrics.std()) * np.sqrt(adj_factor)
        port_metrics_r["Skew"] = port_metrics.apply(skew)
        port_metrics_r["Excess Kurtosis"] = port_metrics.apply(kurtosis, fisher=True, bias=False)
    else:
        asset = 'Portfolio'
        port_metrics_r = pd.DataFrame({
            "Mean": [port_metrics[asset].mean() * adj_factor],
            "Volatility": [port_metrics[asset].std() * np.sqrt(adj_factor)]
        }, index=[asset])
        port_metrics_r["Sharpe_Ratio"] = (port_metrics[asset].mean() / port_metrics[asset].std()) * np.sqrt(adj_factor)
        port_metrics_r["Skew"] = skew(port_metrics[asset])
        port_metrics_r["Excess Kurtosis"] = kurtosis(port_metrics[asset], fisher=True, bias=False)

    # Compute VaR, CVaR, Max Drawdown, and related metrics
    for asset in port_metrics.columns:
        data_aux = port_metrics[[asset]].copy()
        VaR = np.percentile(sorted(data_aux.values.flatten()), q=VaR_q)
        CVaR = data_aux[data_aux[asset] <= VaR].mean().values[0]

        data_aux_acum_return = (data_aux + 1).cumprod()
        data_aux_max_cum_return = data_aux_acum_return.cummax()
        data_aux_drawdown = ((data_aux_acum_return - data_aux_max_cum_return) / data_aux_max_cum_return)
        max_drawdown = data_aux_drawdown.min().values[0]
        max_drawdown_date = data_aux_drawdown.idxmin().values[0]
        peak_idx = data_aux_max_cum_return.idxmax().values[0]
        recovery_data = data_aux_drawdown[data_aux_drawdown.index >= max_drawdown_date]
        recovery_idx = recovery_data[recovery_data[asset] >= -0.00001].first_valid_index()
        duration = (recovery_idx - max_drawdown_date).days if recovery_idx else np.nan

        aux_result = pd.DataFrame(
            [[VaR, CVaR, max_drawdown, max_drawdown_date, peak_idx, recovery_idx, duration]],
            columns=["VaR", "CVaR", "Max Drawdown", "Bottom", "Peak", "Recovery", "Duration (days)"],
            index=[asset]
        )
        result = pd.concat([result, aux_result], axis=0)

    # Merge the two sets of metrics
    metrics = pd.merge(port_metrics_r, result, left_index=True, right_index=True, how="left")
    return metrics

### 2.1.
(8pts)

Calculate the model-implied expected excess returns of every asset.

The time-series estimations should...
* NOT include an intercept. (You assume the model holds perfectly.)
* use data from the `training` timeframe.

With the time-series estimates, use the `training` timeframe's sample average of the factors as the factor premia. Together, this will give you the model-implied risk premia, which we label as
$$
\lambda_i := \mathbb{E}[\tilde{r}_i]
$$

* Store $\lambda_i$ and $\boldsymbol{\beta}^i$ for each asset.
* Print $\lambda_i$ for `Agric`, `Food`, `Soda`

In [7]:
results = pd.DataFrame()

for portfolio in data_sectors_training.columns:
    X = data_factors_training.copy()
    y = data_sectors_training[portfolio].copy()
    mod = sm.OLS(y, X).fit()
    aux = pd.DataFrame(mod.params).T
    aux.index = [portfolio]
    aux["R_squared"] = mod.rsquared
    results = pd.concat([results,aux],axis=0)

In [8]:
results.head()

,MKT,HML,RMW,UMD,R_squared
Agric,0.832362,0.556541,-0.502093,0.038972,0.544657
Food,0.524509,0.205452,0.309711,-0.003572,0.567740
Soda,0.540240,0.179127,0.638443,0.013688,0.536068
Beer,0.541267,0.022959,0.629678,-0.040541,0.613413
Smoke,0.498185,0.443078,0.402220,-0.134015,0.391324


In [ ]:
premium_TS = results[['MKT', 'HML', 'RMW', 'UMD']] @ pd.DataFrame(data_factors_training.mean()*12a)
premium_TS.columns = ["Expected Return TS"]
premium_TS.loc[[ "Agric", "Food ", "Soda "],:]

,Expected Return TS
Agric,0.005415
Food,0.005410
Soda,0.006847


### 2.2.

Use the expected excess returns derived from (2.1) with the **regularized** covariance matrix to calculate the weights of the tangency portfolio.

- Use the covariance matrix only for `training` timeframe.
- Calculate and store the vector of weights for all the assets.
- Return the weights of the tangency portfolio for `Agric`, `Food`, `Soda`.

$$
\textbf{w}_{t} = \dfrac{\tilde{\Sigma}^{-1} \bm{\lambda}}{\bm{1}' \tilde{\Sigma}^{-1} \bm{\lambda}}
$$

Where $\tilde{\Sigma}^{-1}$ is the regularized covariance-matrix.

In [13]:
sigma = data_sectors_training.cov()*12
sigma_reg = (sigma + np.diag(np.diag(sigma)))/2
mu_excess = premium_TS
vector = np.ones(len(mu_excess))
w_tan = (np.linalg.inv(sigma_reg) @ mu_excess )/(np.transpose(vector) @ np.linalg.inv(sigma_reg) @ mu_excess)
weights_db = pd.DataFrame(w_tan)
weights_db.index = data_sectors_training.columns
weights_db.columns = ["w_tan_reg"]
weights_db.loc[[ "Agric", "Food ", "Soda "],:]

,w_tan_reg
Agric,-0.030723
Food,0.015320
Soda,0.132944


### 2.3.

Evaluate the performance of this allocation in the `testing` period. Report the **annualized**
- mean
- vol
- Sharpe

In [14]:
get_metric_returns(data_sectors_testing @ weights_db, adj_factor=12, VaR_q=5)[["Mean", "Volatility", "Sharpe_Ratio"]].style.format("{:.2%}")

,Mean,Volatility,Sharpe_Ratio
w_tan_reg,18.12%,11.95%,151.55%


### 2.4.

(7pts)

Construct the same tangency portfolio as in `2.2` but with one change:
* replace the risk premia of the assets, (denoted $\lambda_i$) with the sample averages of the excess returns from the `training` set.

So instead of using $\lambda_i$ suggested by the factor model (as in `2.1-2.3`) you're using sample averages for $\lambda_i$.

- Return the weights of the tangency portfolio for `Agric`, `Food`, `Soda`.

Evaluate the performance of this allocation in the `testing` period. Report the **annualized**
- mean
- vol
- Sharpe

In [15]:
sigma = data_sectors_training.cov()*12
sigma_reg = (sigma + np.diag(np.diag(sigma)))/2
mu_excess = data_sectors_training.mean()*12
vector = np.ones(len(mu_excess))
w_tan = (np.linalg.inv(sigma_reg) @ mu_excess )/(np.transpose(vector) @ np.linalg.inv(sigma_reg) @ mu_excess)
weights_db_sample_mean = pd.DataFrame(w_tan)
weights_db_sample_mean.index = data_sectors_training.columns
weights_db_sample_mean.columns = ["w_tan_reg"]
weights_db_sample_mean.loc[[ "Agric", "Food ", "Soda "],:]

,w_tan_reg
Agric,0.14409
Food,-0.06981
Soda,0.32268


In [16]:
get_metric_returns(data_sectors_testing @ weights_db_sample_mean, adj_factor=12, VaR_q=5)[["Mean", "Volatility", "Sharpe_Ratio"]].style.format("{:.2%}")

,Mean,Volatility,Sharpe_Ratio
w_tan_reg,17.68%,15.30%,115.55%


### 2.5.

Which allocation performed better in the `testing` period: the allocation based on premia from the factor model or from the sample averages?

Why might this be?

*Answer* : 

The allocation based on premia from the factor model achieved a mean return of 17.56% with a Sharpe Ratio of 156.52%. By comparison, the allocation using the sample mean of the training data produced a similar mean return of 17.68% but with a lower Sharpe Ratio of 115.55%. This indicates that, while both methods yielded comparable average returns, the allocation based on sample averages resulted in higher out-of-sample volatility.

This difference may stem from the nature of the factor model-based allocation, which aims to estimate expected returns more accurately by leveraging relationships identified in the factor model. Assuming the model accurately captures underlying return dynamics, its predictions should theoretically be a more robust estimator for out-of-sample returns than the simple historical average. In contrast, relying solely on sample averages can be prone to higher volatility and may not account for underlying drivers of returns, which can affect out-of-sample performance stability

### 2.6.
Suppose you now want to build a tangency portfolio solely from the factors, without using the sector ETFs.

- Calculate the weights of the tangency portfolio using `training` data for the factors.
- Again, regularize the covariance matrix of factor returns by dividing off-diagonal elements by 2.

Report, in the `testing` period, the factor-based tangency stats **annualized**...
- mean
- vol
- Sharpe


In [53]:
sigma = data_factors_training.cov()*12
sigma_reg = (sigma + np.diag(np.diag(sigma)))/2
mu_excess = data_factors_training.mean()*12
vector = np.ones(len(mu_excess))
w_tan = (np.linalg.inv(sigma_reg) @ mu_excess )/(np.transpose(vector) @ np.linalg.inv(sigma_reg) @ mu_excess)
weights_db_factor_training = pd.DataFrame(w_tan)
weights_db_factor_training.index = data_factors_training.columns
weights_db_factor_training.columns = ["w_tan_reg"]

In [54]:
weights_db_factor_training

,w_tan_reg
MKT,0.176921
HML,-0.016221
RMW,0.598427
UMD,0.240872


In [57]:
get_metric_returns(data_factors_testing @ weights_db_factor_training, adj_factor=12, VaR_q=5)[["Mean", "Volatility", "Sharpe_Ratio"]].style.format("{:.2%}")

,Mean,Volatility,Sharpe_Ratio
w_tan_reg,6.24%,5.82%,107.19%


### 2.7.

Based on the hedge fund's beliefs, would you prefer to use the ETF-based tangency or the factor-based tangency portfolio? Explain your reasoning. Note that you should answer based on broad principles and not on the particular estimation results.

*Answer* :

Based on the hedge fund's beliefs, I would prefer to use the factor-based tangency portfolio. Because it should be more stable in terms that give a better diversification, a factor-based tangency portfolio focuses on underlying risk factors (e.g., market, value, momentum) rather than specific assets. This approach can provide better diversification and robustness, as factors capture common sources of risk that drive asset returns in comparison to use many ETFs for the tangency portfolio which could led to inestability of the covariance matrix (ie. the weights of the portfolio)

***

# 3. Long-Run Returns

For this question, use only the sheet `factors excess returns`.

Suppose we want to measure the long run returns of various pricing factors.

### 3.1.

Turn the data into log returns.
- Display the first 5 rows of the data.

Using these log returns, report the **annualized**
* mean
* vol
* Sharpe


In [61]:
data_factors_log = np.log(1+ data_factors)
data_factors_log.head(5)

,MKT,HML,RMW,UMD
date,,,,
1980-01-01,0.053636,0.017349,-0.017146,0.072786
1980-02-01,-0.012275,0.006081,0.000400,0.075849
1980-03-01,-0.138113,-0.010151,0.014494,-0.100373
1980-04-01,0.038932,0.010544,-0.021224,-0.004309
1980-05-01,0.051263,0.003793,0.003394,-0.011263


In [65]:
metric_1y_log_return = get_metric_returns(data_factors_log, adj_factor=12, VaR_q=5)[["Mean", "Volatility", "Sharpe_Ratio"]]
metric_1y_log_return.style.format("{:.2%}")

C:\Users\josel\AppData\Local\Temp\ipykernel_22068\1304460572.py:63: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, aux_result], axis=0)
C:\Users\josel\AppData\Local\Temp\ipykernel_22068\1304460572.py:63: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, aux_result], axis=0)


,Mean,Volatility,Sharpe_Ratio
MKT,7.35%,15.88%,46.30%
HML,1.98%,10.98%,18.01%
RMW,4.35%,8.36%,52.10%
UMD,5.01%,16.04%,31.22%


### 3.2.

Consider 15-year cumulative log excess returns. Following the assumptions and modeling of Lecture 6, report the following 15-year stats:
- mean
- vol
- Sharpe

How do they compare to the estimated stats (1-year horizon) in `3.1`? 

In [67]:
metric_15y_log_return = metric_1y_log_return.copy()
metric_15y_log_return["Mean"] = metric_15y_log_return["Mean"]*15
metric_15y_log_return["Volatility"] = metric_15y_log_return["Volatility"]*np.sqrt(15)
metric_15y_log_return["Sharpe_Ratio"] = metric_15y_log_return["Mean"]/metric_15y_log_return["Volatility"]
metric_15y_log_return.style.format("{:.2%}")

,Mean,Volatility,Sharpe_Ratio
MKT,110.32%,61.52%,179.33%
HML,29.65%,42.52%,69.75%
RMW,65.31%,32.37%,201.77%
UMD,75.14%,62.14%,120.93%


*Answer:*

Based on the assumptions presented in Lecture 6, expected returns over multiple periods increase proportionally with time. Specifically, over 15 years, the expected return is 15 times the one-year expected return. Meanwhile, the variance of returns increases linearly with time, meaning the standard deviation (the square root of variance) increases with the square root of time. For a 15-year period, the standard deviation is therefore multiplied by √15 compared to the one-year standard deviation.

As a result, the Sharpe Ratio over 15 years—which is calculated as the expected return divided by the standard deviation—becomes the one-year Sharpe Ratio multiplied by √15. This is because the numerator (expected return) is multiplied by 15, while the denominator (standard deviation) is multiplied by √15, leading to an overall multiplication of the Sharpe Ratio by √15.

### 3.3.

What is the probability that momentum factor has a negative mean excess return over the next 
* single period?
* 15 years?

In [72]:
mu, sigma = data_factors_log["UMD"].mean() * 12, data_factors_log["UMD"].std() * np.sqrt(12)
prob_1 = norm.cdf(0, mu, sigma/np.sqrt(1))
prob_15 = norm.cdf(0, mu, sigma/np.sqrt(15))


In [75]:
print(f"Probability of negative return in 1 year: {prob_1:.2%}")
print(f"Probability of negative return in 15 years: {prob_15:.2%}")

Probability of negative return in 1 year: 37.74%
Probability of negative return in 15 years: 11.33%


### 3.4.

Recall from the case that momentum has been underperforming since 2009. 

Using data from 2009 to present, what is the probability that momentum *outperforms* the market factor over the next
* period?
* 15 years?

In [78]:
data_factors_log_present = data_factors_log[data_factors_log.index >= "2009-01-01"]
spread = data_factors_log_present['UMD'] - data_factors_log_present['MKT']
mu, sigma = spread.mean() * 12, spread.std() * np.sqrt(12)

prob_1y = 1 - norm.cdf(0, mu, sigma/np.sqrt(1))
prob_15y = 1 - norm.cdf(0, mu, sigma/np.sqrt(15))

print(f"Probability of positive spread (momentum outperforms the market) in 1 year: {prob_1y:.2%}")
print(f"Probability of positive spread (momentum outperforms the market) in 15 years: {prob_15y:.2%}")

Probability of positive spread (momentum outperforms the market) in 1 year: 28.39%
Probability of positive spread (momentum outperforms the market) in 15 years: 1.35%


### 3.5.
Conceptually, why is there such a discrepancy between this probability for 1 period vs. 15 years?

What assumption about the log-returns are we making when we use this technique to estimate underperformance?

*Answer*:

The discrepancy arises because, under the assumption that log-returns are normally distributed and independent, cumulative log-returns over multiple periods add up linearly, while their standard deviation increases with the square root of time; since the mean spread between Momentum and the Market post-2009 is negative, extending the time horizon amplifies the negative expected return more rapidly than the growth in standard deviation, leading to a much lower probability of outperformance over 15 years (1.35%) compared to 1 year (28.39%).

### 3.6.

Using your previous answers, explain what is meant by time diversification.

*Answer*:

Time diversification means that investing over a longer time horizon can reduce the impact of volatility on the average return due to the Law of Large Numbers; with independent, positive expected returns, the standard deviation of the average return decreases with the square root of time, enhancing the Sharpe Ratio and lowering the risk of underperformance, but in this case, because the expected return (mean spread between Momentum and the Market) is negative, extending the time horizon actually increases the cumulative negative return and the probability of underperformance, illustrating that time diversification can amplify risk when expected returns are negative.

### 3.7.

Is the probability that `HML` and `UMD` both have negative cumulative returns over the next year higher or lower than the probability that `HML` and `MKT` both have negative cumulative returns over the next year?

Answer conceptually, but specifically. (No need to calculate the specific probabilities.)

In [81]:
mu, sigma = data_factors_log["HML"].mean() * 12, data_factors_log["HML"].std() * np.sqrt(12)
prob_1_hml = norm.cdf(0, mu, sigma/np.sqrt(1))

mu, sigma = data_factors_log["UMD"].mean() * 12, data_factors_log["UMD"].std() * np.sqrt(12)
prob_1_umd = norm.cdf(0, mu, sigma/np.sqrt(1))

mu, sigma = data_factors_log["MKT"].mean() * 12, data_factors_log["MKT"].std() * np.sqrt(12)
prob_1_mkt = norm.cdf(0, mu, sigma/np.sqrt(1))

print(f"Probability of negative return in 1 year for HML and UMD: {prob_1_hml * prob_1_umd:.2%}")
print(f"Probability of negative return in 1 year for HML and MKT: {prob_1_hml * prob_1_mkt:.2%}")



Probability of negative return in 1 year for HML and UMD: 16.17%
Probability of negative return in 1 year for HML and MKT: 13.79%


*Answer* : 

The probability that both `HML` (Value factor) and `UMD` (Momentum factor) have negative cumulative returns over the next year is **higher** than the probability that both `HML` and `MKT` (Market) do, because `UMD` has a higher individual probability of negative returns than MKT (momentum has a lower mean and higher variance) so when considering their joint probabilities (assuming independence), the combination of `HML` and `UMD` results in a higher likelihood of both being negative compared to the combination of `HML` and `MKT`.

***